In [20]:
import requests
from google.cloud import bigquery
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
url = 'https://api.binance.com/api/v3/historicalTrades'

r = requests.get(url, params=dict(symbol ="BTCUSDT"))

In [21]:
import pandas as pd
data = r.json()
df = pd.DataFrame(data)
df

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,3477751552,70196.18000000,0.11966000,8399.67489880,1710265314036,True,True
1,3477751553,70196.18000000,0.08579000,6022.13028220,1710265314037,True,True
2,3477751554,70196.15000000,0.01275000,895.00091250,1710265314037,True,True
3,3477751555,70193.33000000,0.00010000,7.01933300,1710265314037,True,True
4,3477751556,70193.33000000,0.00010000,7.01933300,1710265314037,True,True
...,...,...,...,...,...,...,...
495,3477752047,70184.00000000,0.01800000,1263.31200000,1710265322217,True,True
496,3477752048,70184.01000000,0.00022000,15.44048220,1710265322279,False,True
497,3477752049,70184.01000000,0.00597000,418.99853970,1710265322279,False,True
498,3477752050,70184.01000000,0.00153000,107.38153530,1710265322325,False,True


In [23]:
trade_lst = ["BTCUSDT", "AAVEUSDT", "STXUSDT", "ARBUSDT"]
project_id = 'binance-416514'
dwh = 'dwh'
scd = 'scd'
keypath ='/home/tin/credentials/binance-cred.json'

credentials = service_account.Credentials.from_service_account_file(
            keypath, 
            scopes= ["https://www.googleapis.com/auth/cloud-platform"],
            )
# credentials= keypath ,scopes= ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(credentials=credentials)

def extract_data(trade):
    table_id = f'{project_id}.{dwh}.{trade}'

    print(f'now we get {table_id}')
    r = requests.get(url, params=dict(symbol = trade))
    print('getting json')
    data = r.json()
    print('getting dataframe')
    df = pd.DataFrame(data
                    # ,index = [0]
            )
    df['snapshot_date']= datetime.now()

    print(f'start uploading to {table_id}')

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",
    )
    job = client.load_table_from_dataframe(
        df, 
        table_id, 
        job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.
    client.get_table(table_id)

In [25]:
for i in trade_lst:
    extract_data(i)

now we get binance-416514.dwh.BTCUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.BTCUSDT
now we get binance-416514.dwh.AAVEUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.AAVEUSDT
now we get binance-416514.dwh.STXUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.STXUSDT
now we get binance-416514.dwh.ARBUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.ARBUSDT
